In [12]:
import requests
import time
import json
import numpy as np
import blockchain
import BusClass
import queue
import schedule as sc
import json    

In [13]:
with open('data.json') as file:
    data = json.load(file)

In [14]:
target_data = data['feeds'][44:]

In [15]:
q = queue.Queue()
url = "https://api.thingspeak.com/channels/1704499/feeds.json?api_key=Z70IVJ1X27AVPQ6C&results=2"
Rkey = "MC9FFB16PZV2BSWR"
Rkey2 = "MC9GULQTQPUODXP9"
latStation = [7.865661,7.881308,7.893300,7.887864,7.890755,7.882510,7.885414] # lat of stantion point
lonStation = [98.397750,98.364982,98.368747,98.371470,98.390563,98.404646,98.430988] # lon of stantion point
bc = blockchain.Blockchain()# create first blockchain
distanceFormGoal = 0
speed = 0
enitalB1 = 0
enitalB2 = 0
estimateTimeToBusStop = 0

In [16]:
for i in range(len(target_data)):
    q.put(target_data[i])

In [17]:
def getBusStopPoint( stID ) :

    stationId = stID
    point = []

    if stationId == "1.0" :
        point = [latStation[0],lonStation[0]]
    
    elif stationId == "2.0" :
        point = [latStation[1],lonStation[1]]
    
    elif stationId == "3.0" :
        point = [latStation[2],lonStation[2]]
    
    elif stationId == "4.0" :
        point = [latStation[3],lonStation[3]]
    
    elif stationId == "5.0" :
        point = [latStation[4],lonStation[4]]
    
    elif stationId == "6.0" :
        point = [latStation[5],lonStation[5]]
    
    elif stationId == "7.0" :
        point = [latStation[6],lonStation[6]]
        
    else:
        point = ["X","X"]
        
    return point

In [18]:
def addData(data):
    
    baseUrlWr = "https://api.thingspeak.com/update?api_key="
    urlWr = baseUrlWr + Rkey
    dataAdd = ""
    for i in range(len(data)):
        dataAdd = dataAdd + "&field"+ str(i+1) +"=" + str(data[i])
        
    urlWr = baseUrlWr + Rkey + dataAdd

    requests.get(urlWr)
    print("!!! Data Already Add to ThingSpeak Server")
    

In [19]:
def addDataToGPS(data):
    baseUrlWr = "https://api.thingspeak.com/update?api_key="
    urlWr = baseUrlWr + Rkey2
    dataAdd = ""
    for i in range(len(data)):
        dataAdd = dataAdd + "&field"+ str(i+1) +"=" + str(data[i])
        
    urlWr = baseUrlWr + Rkey2 + dataAdd

    requests.get(urlWr)
    print("!!! Data Already Add to GPS Server")

In [20]:
# def readURL():
#     resp = requests.get(url)
#     data_disc = json.loads(resp.text)
#     q.put(data_disc)
#     print("Total queue:" + str(q.qsize()))

In [21]:
# sc.every(25).seconds.do(readURL)

In [22]:
while (bc.is_chain_valid()):   
    
    sc.run_pending()
    
    if(q.qsize() >= 1):
               
        data_disc = q.get()
        busID = data_disc["field5"]
        stationID = data_disc["field4"]
        busStopPoint = getBusStopPoint(stationID)

        if(busStopPoint == ["X","X"]):
            print("Station ID is invalid (1 - 7)")
            break

        if(not ((busID == '1.0')|(busID == '2.0'))):
            print("Bus ID is invalid (1 - 2)")
            break

        if((busID == '1.0') & (enitalB1 == 0)):

            busId1 = BusClass.Bus(float(data_disc["field1"]),float(data_disc["field2"])) 
            distanceFormGoal = busId1.distanceFormGoal(busStopPoint[0],busStopPoint[1])
            speed = busId1.readSpeed()
            enitalB1 = 1;

        if((busID == '2.0') & (enitalB2 == 0)):

            busId2 = BusClass.Bus(float(data_disc["field1"]),float(data_disc["field2"])) 
            distanceFormGoal = busId2.distanceFormGoal(busStopPoint[0],busStopPoint[1])
            speed = busId2.readSpeed()
            enitalB2 = 1;

        if((busID == '1.0') & (enitalB1 > 1)):

            busId1.setBusLocation(float(data_disc["field1"]),float(data_disc["field2"]))
            distanceFormGoal = busId1.distanceFormGoal(busStopPoint[0],busStopPoint[1])
            speed = busId1.readSpeed()

        if((busID == '2.0') & (enitalB2 > 1)):

            busId2.setBusLocation(float(data_disc["field1"]),float(data_disc["field2"]))
            distanceFormGoal = busId2.distanceFormGoal(busStopPoint[0],busStopPoint[1])
            speed = busId2.readSpeed()

        if((busID == '1.0') & (enitalB1 == 1)):

            enitalB1 = 2

        if((busID == '2.0') & (enitalB2 == 1)):

            enitalB2 = 2
            
        estimateTimeToBusStop = (distanceFormGoal/speed) * 60

        bc.mine_block("Bus ID :" + str(busID) + 
                      ",Station ID :" + str(stationID) + 
                      ",Distance to bus stop :" + str(distanceFormGoal) +
                      ",Speed :" + str(speed) + 
                      ",Estimate time to bus stop :"+ str(estimateTimeToBusStop)
                     )
        dataForAdd = [bc.chain[len(bc.chain)-1]["index"],
                     bc.is_chain_valid(),
                     busID,
                     stationID,
                     distanceFormGoal,
                     speed,
                     estimateTimeToBusStop,
                     bc.chain[len(bc.chain) - 1]["previous_hash"]]
        
        dataForAddGPS = [data_disc["field1"],
                         data_disc["field2"],
                         data_disc["field2"],
                         data_disc["field4"],
                         data_disc["field5"]]

        addData(dataForAdd)
        addDataToGPS(dataForAddGPS)

        print(bc.chain[len(bc.chain)-1])# print the current block

    time.sleep(15)

C:\Users\ziver\AppData\Local\Temp\ipykernel_7864\2931484985.py:54: RuntimeWarning: divide by zero encountered in double_scalars
  estimateTimeToBusStop = (distanceFormGoal/speed) * 60


!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 2, 'timestamp': '2022-11-10 21:35:35.795584', 'data': 'Bus ID :1.0,Station ID :1.0,Distance to bus stop :5.23073190165754,Speed :0.0,Estimate time to bus stop :inf', 'proof': 2303, 'previous_hash': '770cb8a9f8d8979b407e618345b85a6aa63c02c7443e7095923671ea6c74cf53'}
!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 3, 'timestamp': '2022-11-10 21:35:53.159803', 'data': 'Bus ID :2.0,Station ID :1.0,Distance to bus stop :4.945377422482968,Speed :0.0,Estimate time to bus stop :inf', 'proof': 5612, 'previous_hash': '4dda507f816fb9446500d61b1206d4f9d63c407b8008f328c7c58d9c8cbce4a4'}
!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 4, 'timestamp': '2022-11-10 21:36:10.541953', 'data': 'Bus ID :2.0,Station ID :1.0,Distance to bus stop :4.9725529786493485,Speed :35.81647505554399,Estimate time to bus stop :8.330054207073044', 'proof': 9

!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 24, 'timestamp': '2022-11-10 21:41:58.044315', 'data': 'Bus ID :2.0,Station ID :1.0,Distance to bus stop :2.6013529884928364,Speed :19.237152805111418,Estimate time to bus stop :8.113528071997148', 'proof': 636, 'previous_hash': '2b74724d089ef0b05212336f031b263574fdf8a84a5982bf246b977e72edb097'}
!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 25, 'timestamp': '2022-11-10 21:42:15.387828', 'data': 'Bus ID :2.0,Station ID :1.0,Distance to bus stop :2.5839431922341194,Speed :13.079787795274688,Estimate time to bus stop :11.853142723772395', 'proof': 2288, 'previous_hash': '0a3c49a013a6c63cb414e82d2446ca0e53ca42f34ea85e111ac9ac8e8d2063de'}
!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 26, 'timestamp': '2022-11-10 21:42:32.782361', 'data': 'Bus ID :2.0,Station ID :1.0,Distance to bus stop :2.513825352222178,Speed :7.2230786835

!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 46, 'timestamp': '2022-11-10 21:48:20.529682', 'data': 'Bus ID :1.0,Station ID :5.0,Distance to bus stop :2.1143907994524516,Speed :6.907454914826716,Estimate time to bus stop :18.366163736347694', 'proof': 5261, 'previous_hash': '3bb9a7f2d508339a2ee49a39bf5a63e02aacbc02becae279e716449575b9e9da'}
!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 47, 'timestamp': '2022-11-10 21:48:37.932730', 'data': 'Bus ID :1.0,Station ID :5.0,Distance to bus stop :2.2117435761996616,Speed :9.313833618567498,Estimate time to bus stop :14.248119518414818', 'proof': 4508, 'previous_hash': 'ba3e7b4c2fef39f5cce0235061dcccfd3ae6ff4ad38085dd5db0a88896909bd2'}
!!! Data Already Add to ThingSpeak Server
!!! Data Already Add to GPS Server
{'index': 48, 'timestamp': '2022-11-10 21:48:55.321376', 'data': 'Bus ID :1.0,Station ID :5.0,Distance to bus stop :2.4021499568851117,Speed :12.25026843

KeyboardInterrupt: 